# High-performance computing 2023/24

- Name: Sascha Petznick (6106749)
- Mail: S.C.Petznick@student.tudelft.nl
- Date of Submission: 21.12.2023

## Overview

- PingPong
- MM-Product
- Poisson solver
- Something else

• A final report on the lab exercises must be submitted which contains all
your answers to the questions in the three exercises.
• Any figures, analysis, and tools, illustrating or demonstrating your
answers,will be beneficial for your overall grades.
• The final report should be completed independently. It should have the
following information on the front cover: your name(s), studentnumber(s),
email address(s), and submission date.
• __Submit your report to Lab reports in Assignments on Brightspace, please also
print your reports (no tedious source code) and hand in it to TA T. Deng. After
the submission please send us an email. This is to make sure that your report
is not unexpectedly got lost.__
• Submission Deadline: February 1st 2023

In [1]:
import matplotlib as pyplot
import numpy as np

# consider using https://github.com/kirbs-/hide_code to hide specific cells or 
# use second python script for creating plots

## PingPong

Messages of sizes $2^n$ where $n=0,\dots,20$ were sent between two MPI nodes. The time needed to send the message to node two, receive it there, send it back to node one, and receive it there was measured. Every message size was sent five times and the time was averaged. Moreover, the test were conducted on the supercomputer using one computing node with two processes and two computing nodes in order to analyze the difference.

The results are displayed in plots in Figure ([1](#communication-time-over-message-size)) and ([2](#communication-time-over-message-size)). Based on visual comparison it is clear that using two node has a minor effect on the experiments. Thus, the following analysis will be based on the experiment with one node.

Assuming that the message size $t_{\text{comm}}(m) = \alpha + \beta \cdot m$ is in seconds, we approximate the parameters using a linear regression as $(\alpha, \beta) = (6.49\cdot 10^{-6} [s], 5.713\cdot 10^{-9} [s/\text{bytes}])$ as opposed to the exercise sheet. The linear regression was done by dropping the first measurement in order to reduce the effect of the start-up process of MPI. The regression is displayed on Figure (1). It should be mentioned that the plots are displayed in double-logarithmis scaling and the author decided against using two linear regressions and instead used a single one as the logarithmic scaling helps identifying the overall trend and accuracy of the regression.

#### Communication time over message size

![FigureOfCommTimeOverMessageSizeN1](./figs/pingpong-doublelog_n1.svg)

![FigureOfCommTimeOverMessageSizeN2](./figs/pingpong-doublelog_n2.svg)



Based on the visualization, it is clear that the experiment for small message sizes suffers from a higher variance. This may be caused by the overall initialisation overhead that is assumed to play minor role when the message size is increased. Doing so, yields a clear trend of the function towards a linear dependence. One may argue that increasing the message size above magnitudes of $10^4$ bytes requires a second trending. Overall, this may come down to style. A possible reason for the visible jump in times when increasing the message size above $16.384$ bytes is the general size of the buffer of the MPI communicator. It might well be that messages of this size require several messages to be passed instead of fitting into a single buffer.

## MM-Product

Let $A, B \in \mathbb{R}^{n\times n}$.  

### Main idea of distributing the computations

The distribution of the matrices is as follows. We distribute the matrix $A$ to all processes using the broadcast operation of MPI. In addition, we scatter rows of $B$ in a blocksize of $\lfloor n/p\rfloor$ where $p$ is the number of processes. If it so happens that the rows is not divisible by the number of processes, the last process will work on the rest of the rows that remain when using the blocksize for all the other processes.
The following code snippet gives a general idea of the communication that is used ignoring setup and finalisation of the MPI process.

```[C]
MPI_Bcast(a, n * n, MPI_DOUBLE, 0, MPI_COMM_WORLD);
MPI_Scatterv(b, counts, displs, MPI_DOUBLE, b_local, counts[rank],
    MPI_DOUBLE, 0, MPI_COMM_WORLD);


MPI_Gatherv(c_local, counts[rank], MPI_DOUBLE, c, counts, displs,
    MPI_DOUBLE, 0, MPI_COMM_WORLD);
```

In order to optimise the speed of the computations on each node, the matrices $B$ and $C$ are stored in aligned memory in a column-major fashion, which simplifies the calls to `MPI_Scatterv` and `MPI_Gatherv`.

### Plots

For the analysis of the matrix multiplication algorithm, the program was run five times for matrix sizes $10\times 10$, $100\times 100$, and $1000\times 1000$ in a single go where after each iteration allocated memory was freed. 
Overall, the program was instructed using 1, 2, 8, 24, 48, and 64 computing nodes in order to analyse the effect of an increasing number processes. The timing was started before initialising any matrix including the given matrices $A$ and $B$ of size $n$ as well as the result matrix.

Although runs with matrix size of less than 1000 benefitted from increasing the number of computing nodes, we focus on the computations done on matrix size $1000 \times 1000$. It should be mentioned that increasing the size to even larger matrices would required the program to handle matrices that were rejected by the memory handler of DelftBlue and therefore would require different approaches. Matrices of row size 1000 require about 8MB of memory and fit easily into RAM.

Figure ([3](#times-of-run-over-the-number-of-processes)) displays the times needed for each iteration. The time required for the computation appears to reduce by increasing the number of computing nodes. The benefit becomes marginal as the number of processes increases beyond 24 across all sizes of the matrix. However, the average times of the matrix multiplication of sizes 100 using 64 nodes turned out to be slower. This is caused an implementation detail: If the number of nodes is larger than half of the number of rows of the matrices, then the last computing node is required to compute many rows. In this case 37 rows which increases the overall running time significantly. The proposed implementation therefore relies on the assumption that twice as many rows than computing nodes exist in order to distribute the work evenly. This assumption is generally applicable to real-world examples where the size of the matrix is much larger than the number of available computing nodes.   

Although not clearly visible in the figure, the first run often took longer that subsequent runs with the same matrix sizes. As the allocated memory of the implemented code is freed after each run we can exclude allocation of memory on the user side from the set of possible explanations if we assume that the operating system has sufficient memory to provide subsequent calls with the same amount of memory. As the program was instructed to require 1GB of RAM and the allocated memory for matrices sums up to around 200MB, this seems to be reasonable. However, the internal buffer size of the MPI operations, in this case Broadcast and Scatter, may require the MPI subprocess to increase the buffer for larger matrices. As the slowdown only happened on the first calls after increasing the matrix sizes this explanation also fits the general experimental setup.
Moreover, some runs of matric sizes 10 and 100 took longer than the job of matrix size 1000. For example, for 24 nodes some two runs were slower. Overall, the trend follows the theoretical idea of reducing the computing time when increasing the number of nodes at least if the matrix size is reasonable large enough. Unnecessarily large computing times for small matrices may be circumvented by falling back to an algorithm that operates only on one node.  



#### Times of run over the number of processes

![FigureOfTimeOverNumProcesses](./figs/MM-product-time-over-procs.svg)

## Exercise 1: Parallel solver for the Poisson equation

### 1.2.2

Running the problem several times with a topology of 4 x 1 yields and optimal value of around 1.93 as suggested by the exercise. For values $\omega \in [1.9,1.99]$ we did a binary-like search in order to find this optimal value. The number of iterations using the optimal value turned out to be $131$ taking around $0.042 [s]$.

precision goal: 0.000100
Number of iterations: 131, omega: 1.93
Delta: 0.000066
(0) Elapsed Wtime       0.041549 s ( 21.5% CPU)
precision goal: 0.000100
Delta: 0.000089
(1) Elapsed Wtime       0.013035 s ( 82.9% CPU)
precision goal: 0.000100
Delta: 0.000099
(2) Elapsed Wtime       0.077678 s ( 14.0% CPU)
precision goal: 0.000100
Delta: 0.000078
(3) Elapsed Wtime       0.057615 s ( 15.5% CPU)


- 4 1 1.93 100x100
Number of iterations: 131, omega: 1.93
Delta: 0.000066
Delta: 0.000089
Delta: 0.000099
Delta: 0.000078
(1) Elapsed Wtime       0.005309 s ( 89.1% CPU)
(2) Elapsed Wtime       0.005308 s ( 85.5% CPU)
(3) Elapsed Wtime       0.005310 s ( 86.7% CPU)
(0) Elapsed Wtime       0.009293 s ( 89.2% CPU)

- 2 2 1.93 100x100
Number of iterations: 131, omega: 1.93
Delta: 0.000089
Delta: 0.000082
Delta: 0.000099
Delta: 0.000087
(1) Elapsed Wtime       0.005498 s ( 89.0% CPU)
(2) Elapsed Wtime       0.005496 s ( 89.0% CPU)
(3) Elapsed Wtime       0.005501 s ( 90.1% CPU)
(0) Elapsed Wtime       0.009459 s ( 90.4% CPU)

- 4 1 1.93 200x200
Number of iterations: 532, omega: 1.93
Delta: 0.000063
Delta: 0.000098
Delta: 0.000099
Delta: 0.000078
(1) Elapsed Wtime       0.059708 s ( 94.3% CPU)
(2) Elapsed Wtime       0.059707 s ( 94.5% CPU)
(3) Elapsed Wtime       0.059712 s ( 94.2% CPU)
(0) Elapsed Wtime       0.074123 s ( 94.9% CPU)

- 2 2 1.93 200x200
Number of iterations: 532, omega: 1.93
Delta: 0.000098
Delta: 0.000096
Delta: 0.000099
Delta: 0.000098
(1) Elapsed Wtime       0.060286 s ( 98.1% CPU)
(2) Elapsed Wtime       0.060288 s ( 97.5% CPU)
(3) Elapsed Wtime       0.060292 s ( 97.8% CPU)
(0) Elapsed Wtime       0.075101 s ( 97.1% CPU)

- 4 1 1.93 400x400
Number of iterations: 1561, omega: 1.93
Delta: 0.000066
Delta: 0.000099
Delta: 0.000100
Delta: 0.000077
(1) Elapsed Wtime       0.619254 s ( 98.8% CPU)
(2) Elapsed Wtime       0.619255 s ( 99.0% CPU)
(3) Elapsed Wtime       0.619252 s ( 98.9% CPU)
(0) Elapsed Wtime       0.673155 s ( 99.0% CPU)

- 2 2 1.93 400x400
Number of iterations: 1561, omega: 1.93
Delta: 0.000099
Delta: 0.000098
Delta: 0.000100
Delta: 0.000099
(1) Elapsed Wtime       0.620277 s ( 99.3% CPU)
(2) Elapsed Wtime       0.620275 s ( 99.1% CPU)
(3) Elapsed Wtime       0.620277 s ( 99.0% CPU)
(0) Elapsed Wtime       0.674249 s ( 99.3% CPU)

- 4 1 1.93 800x800
Number of iterations: 3601, omega: 1.93
Delta: 0.000071
Delta: 0.000100
Delta: 0.000100
Delta: 0.000072
(1) Elapsed Wtime       5.472918 s ( 99.7% CPU)
(2) Elapsed Wtime       5.472918 s ( 99.7% CPU)
(3) Elapsed Wtime       5.472930 s ( 99.7% CPU)
(0) Elapsed Wtime       5.691494 s ( 99.7% CPU)

- 2 2 1.93 800x800
Number of iterations: 3601, omega: 1.93
Delta: 0.000100
Delta: 0.000100
Delta: 0.000100
Delta: 0.000100
(1) Elapsed Wtime       5.486153 s ( 99.7% CPU)
(2) Elapsed Wtime       5.486152 s ( 99.8% CPU)
(3) Elapsed Wtime       5.486152 s ( 99.8% CPU)
(0) Elapsed Wtime       5.705497 s ( 99.7% CPU)





### 1.2.3

Running the solver five times for grid sizes 100, 200, 400, and 800 and measuring the overall time and computing the time per iteration yields the times as displayed in the plot below. A value $\omega=1.95$ was used.

![FigureBoxPlotOfTimePerIterationOverGridSizes](./figs/exercise01_ex_1_2_3.svg)


As can be seen from the from the plots, there is no clear dominating topology that is better than the others.
In general, the analysis and the presentation of data would have benefitted from running the experiments more often than five times as some boxplots suffer from a dominance of outliers.

There are arguments that support these finding and arguments against this.
One may argue that the 2x2 topology requires to exchange four borders whereas the 4x1 and 1x4 topologies only require to exchange two borders with actual data transfer. The other call to communication will no lead to any actual computation as there is no border.
Although in all cases the overall number of data points to exchange remains about the same (ignoring corners of each local grid that need to be communicated double) one may expect an overall decrease of time as two calls to the corresponding MPI functions may run faster than four calls when both communicate the same amount of data.
However, the practical analysis clearly shows that the grid sizes do not enable us to make any predictions about the scaling with respect to the topology. This stems from the fact that time for communication with these sizes is still negligible and therefore has a minor affect as later analysis of the communication times will show. 

As can be seen in [section 1.2.5](#1.2.5) the grid size may no be the best predictor for overall number of iterations (and thus for the time spent). For the sake of the exercise, we still compute a linear regression on the 800x800 topologies by merging all experiments on that grid.

The formula is computed to be: $t(n) = 0.167 [ms] + 2.22 \cdot n [ms/iteration]$

### 1.2.4

As already pointed out in the previous section, it is difficult to make any prediction about which topology would dominate in performance when increasing the number of processes given the data that was gathered.



## 1.2.5

The results of the experiments regarding the number of iterations until convergence is reached are presented in the table below.

| Topology | $\omega$ | Grid Size | Number of Iterations |
|----------|----------|-----------|-----------------------|
| 4 1      | 1.93     | 100x100   | 131                   |
| 2 2      | 1.93     | 100x100   | 131                   |
| 4 1      | 1.93     | 200x200   | 532                   |
| 2 2      | 1.93     | 200x200   | 532                   |
| 4 1      | 1.93     | 400x400   | 1561                  |
| 2 2      | 1.93     | 400x400   | 1561                  |
| 4 1      | 1.93     | 800x800   | 3601                  |
| 2 2      | 1.93     | 800x800   | 3601                  |
| 4 1      | 1.93     | 1600x1600 | 3981                  |
| 2 2      | 1.93     | 1600x1600 | 3981                  |

The trend is appears linear until 800x800 as can be seen from the following graphic. However, the number of iterations for 1600x1600 stays below 4000. It seems as if the grid size may be a bad predictor estimating the number of iterations to reach convergence. It should be mentionend that the stopping criterion leads to very different solution for different grid sizes as information propagation is slower on finer grids. 

The maximum error which is used as a stopping criterion is mainly determined by points close to sources and the error is reduced relatively fast no matter the size of the grid. However, points that a further away from sources will generally attain values of lower magnitude and therefore have no impact on the maximum error that is used.

![FigureOfNumIterationsOverGridSize](./figs/exercise01_ex_1_2_5.svg)

### 1.2.6

The global error over the number of iterarion is displayed below.

![FigureOfGlobalErrorOverNumIterations](./figs/exercise01_ex_1_2_6.svg)

## 1.2.7

It was found that there was virtually no increase in performance when restricting the collective communication to every 10th iteration. This appears to relate to the fact that the time spent in `MPI_Allreduce` is less than 0.2% in the case of an 800x800 grid on a topology of 2x2 using $\omega = 1.93$. The time spent was still reduced to less than 0.1%.

## 

## 1.2.8

Running the experiment on a 2x2 topology using the 800x800 grid, we were able to identify no real progress at all using the optimal omega. However, as can be seen in the table, the resulting solution starts to improve when the number of sweeps is increased above 17. Although the solutions has still no met the converging criterion increasing the number of sweeps may be beneficial. Notice that the computation of iterations as provided in the table is the number of iterations divided by the number of sweeps per iteration in order to have comparable results with the single-sweep solution. Please also notice that the number of iterations needed on the 800x800 grid is lower when using a different omega ($\omega=1.93$: 3600 to $\omega=1.4$: 2769).

| Number of Sweeps | Number of Iterations | Delta                    |
|-------------------|----------------------|--------------------------|
| 1                 | 3601                 | 0.0001                   |
| 2                 | 5000                 | 0.00011                  |
| 4                 | 5000                 | 0.000183                 |
| 5                 | 1000                 | Diverged                 |
| 8                 | 625                  | Diverged                 |
| 11                | 455                  | Diverged                 |
| 14                | 358                  | Diverged                 |
| 17                | 295                  | Diverged but low 0.005044                 |
| 20                | 250                  | Diverged but low 0.000709                 |
| 23                | 218                  | Diverged but low 0.000651                 |
| 26                | 193                  | Diverged but low 0.000655                 |



# 1.2.9

In order to analyse the reduction in overall running time, experiments with the solver with and without parity check were conducted. The times of each processor were averaged and in every experiment the first run was neglected due to the offset in timing.

As displayed below, the parity check has a significant impact on the running time of the algorithm.
The overall time is decreased by about 20% which is significant.

![FigureOfRunTimeComparisonParity](./figs/exercise01_ex_1_2_9.svg)

# 1.2.10 and 1.2.11

The amount of data that is communicated during a single call of `exchange_borders` can be computed using the size of the `MPI_Datatypes` that are used for the exchange, the number of processors, and the topology which they are allocated in.
Let the total number of processors in use be $n$. In a row-only or column-only topology, i.e. where one dimension of the topology is a 1, $n-2$ processors communicate 
$$ 2\cdot 8 [Bytes] \cdot n_i \cdot n_{\mathrm{iterations}}$$

where $n_i$ is the number of grid points that is along the axis of communication and $n_{\mathrm{iterations}}$ is the number of iterations.
The remaining two processes only communicate the half of this, as they touch the boundary on all other sides.

For the case of different topologies that the two aforementioned the amount of data to be communicated extends to

$$ 4\cdot 8 [Bytes] \cdot n_i \cdot n_{\mathrm{iterations}}$$

## 1.2.12

Adapting the routine to exchange the borders may improve the algorithm if communication time consumes a bigger part of the overall run time. As of now, this was not the case. However, implementation of an improved border exchange scheme can be done by changing the MPI type vectors such that they only point to black or red points and thus only update the required points during one red- or black-sweep. The first points to exchange varies depending on the parity which increases the complexity of the code quite a lot. The number of points to communicate drops to a half, from $2n/p$ to $n/p$ per side per number of processors $p$. Using this approach, the memory of the data is still aligned as previously which requires to change the stride of the MPI data types as well.
As the communication time has not consumed a big chunk of the overall run time, we advice against the implementation.
However, the idea to work with separate data structures for red- and black-points may reduce the complexity of the code. Furthermore, this aids alignment of memory as communication can partially use aligned memory as before.

## 1.2.13

Changing the algorithm improves the speed tremendously. In the following plot, we displayed the results of the experiments using the conjugate-gradient method. The time per iteration was reduced by 2 orders of magnitude which renders larger sizes of $n=1600$ feasible on the hardware.

![FigureBoxPlotOfTimePerIterationOverGridSizes](./figs/exercise01_ex_1_2_13.svg)